## OneAtlas data search for multiple aoi's in folder with search report, coverage and metadata output

### Libraries

In [ ]:
import requests
from pathlib import Path
import json
import geopandas as gp
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from datetime import datetime
from datetime import timedelta
import os
import fiona
import csv

In [ ]:
#fiona.supported_drivers

### Credentials

In [ ]:
# PROJECT CREDENTIALS

# Path, load read json with credential secrets
proj_conf_file_path = Path.cwd()/'project-config-file.json'

# Opening JSON file and load into dict
open_proj_conf_file = open(proj_conf_file_path)
project_credentials = json.load(open_proj_conf_file)

In [ ]:
# PROJECT CREDENTIALS

client_id = project_credentials['project_id']
client_secret = project_credentials['project_api_key']

### Functions

In [ ]:
# Get access token
def get_ptoken(client_id, client_secret):
    response = requests.post("https://api.up42.com/oauth/token",
                             data={"grant_type": "client_credentials"},
                             auth=(client_id, client_secret)
                            )
    
    return response.json()["access_token"]

# Get all hosts
def get_list_hosts(PTOKEN):
    response = requests.get("https://api.up42.com/hosts",
                            data={"grant_type": "client_credentials"},
                            headers={"Authorization": "Bearer " + PTOKEN}
                           )
    return response.json()["data"]

# Search data through Hosts 
def search_hosts(PTOKEN, HOST_NAME):
    response = requests.post("https://api.up42.com/catalog/hosts/"+HOST_NAME+"/stac/search",
                            headers={"Authorization": "Bearer " + PTOKEN,
                                    "Content-Type" : "application/json"},
                             data=updated_payload
                           )
    return response.json()

### Available Hosts

In [ ]:
# Build our host list calling "build_host_list()" function

PTOKEN = get_ptoken(client_id, client_secret)
#PTOKEN

hosts_name = get_list_hosts(PTOKEN)
current_host_list = [((hosts_name)[i]['name']) for i in range(len(hosts_name))]

In [ ]:
current_host_list

### Output folder and AOI folder path

In [ ]:
# Create an folder to save the search results into
time_stamp = datetime.now().strftime("%Y%m%d%H%M")
out_dir_name = './search_results_clt_'+time_stamp+'/'
os.mkdir(out_dir_name)

In [ ]:
# AOI folder path
aoi_folder_path = Path.cwd()/'aoi/columbia_land_trust'

### Run the search over the aoi's in folder

In [ ]:
# list to hold basic search result information about each aoi to a csv for easy for quick look at data availability over each aoi
csv_data = []

# Loop and script
for geojson_file in sorted(aoi_folder_path.iterdir()):
    if geojson_file.suffix == ".geojson":

   # Open and load an aoi file     
        aoi_file = open(geojson_file, encoding = "ISO-8859-1")
        aoi = json.load(aoi_file)
        intersects_coord = aoi['features'][0]['geometry']

#        print(f'{geojson_file.stem} - {intersects_coord}')

    # Search parameters payload template
        file_path = Path.cwd()/'search_params/sr_land_trust.json'

    # Opening payload file and load into dict
        sr_file = open(file_path)
        payload = json.load(sr_file)
#        print(payload)

    # Update the search parameters payload with the current aoi coordinates
        #del payload["intersects"]
        payload["intersects"] = intersects_coord

        updated_payload = json.dumps(payload)
        print(updated_payload)

    # Search the archive
        PTOKEN = get_ptoken(client_id, client_secret)
        HOST_NAME = "oneatlas"
        gp_search_result = gp.GeoDataFrame.from_features(search_hosts(PTOKEN, HOST_NAME))
       # gp_search_result
    
    # Number of scenes found
        print(f"{geojson_file.stem} - {len(gp_search_result['collection'])} {HOST_NAME} scenes found")

        
    # Break down resuls for use in csv later
        results_for_csv = gp_search_result['collection'].value_counts()
        
        try:
            pneo_result_count = results_for_csv['pneo']
        except:
            pneo_result_count = 0
        
        try:
            pleiades_result_count = results_for_csv['phr']
        except:
            pleiades_result_count = 0
            
        try:    
            spot_result_count = results_for_csv['spot']
        except:
            spot_result_count = 0        
        
        csv_data.append([geojson_file.stem, aoi['features'][0]['properties']['id'], pneo_result_count, pleiades_result_count, spot_result_count])
        
          
    # remove list from the geopandas dataframe or will not be able to save file geojson
        drop_column_gp = gp_search_result.drop(columns=['up42:usageType'])
        
    # CSet output file names (result metadata and coverage)
        write_file_name = geojson_file.stem+'_results'
        output_path_name = './'+out_dir_name+'/'+write_file_name+'.geojson'
        
    # Write results to geojson
        drop_column_gp.to_file(output_path_name, mode='w', driver='GeoJSON')
        
        
        

### Build and save CSV file containing basic search result information per aoi

In [ ]:
# Print the data that will go into the csv file
#csv_data

In [ ]:
# Build csv from results in list csv_data
csv_output_path = './'+out_dir_name+'/clt_oneatlas_search_results.csv'

header = ['aoi_file', 'id', 'pneo', 'pleiades', 'spot']

with open(csv_output_path, 'w', encoding='UTF8') as f:
    writer = csv.writer(f)
           
    # write the header
    writer.writerow(header)    
            
    # write the data
    writer.writerows(csv_data)  

If you have any questions or would like advice on other up42 things, feel free to reach out!!
Happy developing - Alex (alex.bishop@up42.com)